### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [32]:
%load_ext autoreload
%autoreload 2

In [82]:
from tqdm import tqdm
from collections import namedtuple
from sklearn.neighbors import KDTree 
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [4]:
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python', encoding='latin-1')

In [6]:
movie_info.head()

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Explicit данные

In [7]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [8]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [9]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [10]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [11]:
users

0             1
3             1
4             1
6             1
7             1
           ... 
1000202    6040
1000205    6040
1000206    6040
1000207    6040
1000208    6040
Name: user_id, Length: 575281, dtype: int64

In [12]:
movies

0          1193
3          3408
4          2355
6          1287
7          2804
           ... 
1000202    1089
1000205    1094
1000206     562
1000207    1096
1000208    1097
Name: movie_id, Length: 575281, dtype: int64

In [13]:
user_item

<6041x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 575281 stored elements in COOrdinate format>

In [14]:
user_item_csr

<6041x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 575281 stored elements in Compressed Sparse Row format>

In [15]:
user_item_csr.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
###
_us = np.array([0, 0, 1, 2])
_it = np.array([0, 3, 5, 7])
_us_it = sp.coo_matrix((np.ones_like(_us), (_us, _it)))
_us_it.toarray()

array([[1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [17]:
###

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [18]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [19]:
model.fit(user_item_t_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [20]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [22]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [23]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [24]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [25]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [26]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '1179    Princess Bride, The (1987)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [94]:
EPOCHS = 10

In [93]:
from models import SGD_bias
from RecSys import build_recsys

In [114]:
sgd_model = SGD_bias(rank=64, lr=1e-2, lambd=1e-2, eps=1e-3, max_epochs=EPOCHS)
sgd_model.fit(ratings)

Users: 6040 | Movies: 3706 | Ratings: 1000209


100%|██████████| 1000209/1000209 [02:22<00:00, 7017.95it/s]


Epoch: 0 | MSE_mean=0.9065413843754566


100%|██████████| 1000209/1000209 [02:22<00:00, 7031.97it/s]


Epoch: 1 | MSE_mean=0.8258805495441682


...

Epoch: 7 | MSE_mean=0.6564743291098125


100%|██████████| 1000209/1000209 [02:08<00:00, 7769.97it/s]


Epoch: 8 | MSE_mean=0.6196712560869547


100%|██████████| 1000209/1000209 [02:08<00:00, 7788.68it/s]

Epoch: 9 | MSE_mean=0.5840352762079175


In [63]:
SGD_RECSYS = build_recsys(ratings, sgd_model)

In [64]:
get_similars(1, SGD_RECSYS)

['3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '2012    Little Mermaid, The (1989)',
 '2021    Rescuers, The (1977)',
 '1526    Hercules (1997)',
 '591    Beauty and the Beast (1991)',
 '1838    Mulan (1998)',
 '1664    Mouse Hunt (1997)',
 '1016    Dumbo (1941)']

In [65]:
get_recommendations(4, SGD_RECSYS)

['900    Casablanca (1942)',
 '2836    Sanjuro (1962)',
 '1189    To Kill a Mockingbird (1962)',
 '2953    General, The (1927)',
 '1950    Seven Samurai (The Magnificent Seven) (Shichin...',
 '1194    Third Man, The (1949)',
 '3269    For All Mankind (1989)',
 '896    North by Northwest (1959)',
 '892    Rear Window (1954)',
 '1186    Lawrence of Arabia (1962)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [108]:
from models import ALS

In [110]:
als_model = ALS(rank=64, confidence=1e4, lambd=1e-2, eps=1e-3, max_epochs=EPOCHS)
als_model.fit(user_item_csr)

Users: 6038 | Movies: 3533 | Ratings: 575281


100%|██████████| 6038/6038 [00:05<00:00, 1007.49it/s]


Epoch: 1 | MSE=0.06493975947875012


100%|██████████| 3533/3533 [00:10<00:00, 328.88it/s]


Epoch: 2 | MSE=0.0002035355328434781


In [111]:
ALS_RECSYS = build_recsys(ratings, als_model)

In [112]:
get_similars(1, ALS_RECSYS)

['1245    Groundhog Day (1993)',
 '3045    Toy Story 2 (1999)',
 '584    Aladdin (1992)',
 '360    Lion King, The (1994)',
 '148    Apollo 13 (1995)',
 '1250    Back to the Future (1985)',
 '33    Babe (1995)',
 '591    Beauty and the Beast (1991)',
 '1246    Unforgiven (1992)',
 '1120    Monty Python and the Holy Grail (1974)']

In [113]:
get_recommendations(4, ALS_RECSYS)

['1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 "1176    One Flew Over the Cuckoo's Nest (1975)",
 '900    Casablanca (1942)',
 '3509    Gladiator (2000)',
 "887    Singin' in the Rain (1952)",
 '912    2001: A Space Odyssey (1968)',
 '1239    Stand by Me (1986)',
 '2502    Matrix, The (1999)']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных